<h1> Data Pre-processing</h1>
<h3>Title:</h3> <em>Processing data on US Accidents Dataset for visualizations and preparation of data for training and testing the models for predictions</em> 
<h3>Author:</h3> <em>Uttam Kumar</em>

<em> Firstly for doing exploratory data analysis and then for building train and test sets of models </em>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import regex as re
import warnings; warnings.simplefilter('ignore')
from feature_extraction import *

In [2]:
#arp_df = pd.read_csv('../data_files/US_Accidents_June20.csv')  #only for first time when no modified version exist
#arp_df = pd.read_pickle('../data_files/US_Accidents_June20_modified.pkl') #3513617 rows, 57 col incl cust time 
#arp_df = pd.read_pickle('../data_files/US_Accidents_June20_modified_v1.pkl') #geo features, 3513617 row, 56 clmn 
#arp_df = pd.read_pickle('../data_files/US_Accidents_June20_modified_v2.pkl') #weather cond., 3513617 row, 69 clmn
#NaN handled, 3487860 rows, 70 colmns, then #4 severity, 4 timezones, 2 side, 8 day/night, 18 wind_direction 
#i.e. 106 cols, 3487859 rows, 1 r del as Side = ' ', 22 TMC columns and 39 colmns of weather params, i.e. 160 colm 
arp_df = pd.read_pickle('../data_files/US_Accidents_June20_modified_v3.pkl') 
#Comn strt 25, 1+96 timeQuartOfDay, i.e. total 282 columns,  3487839 rows exist now
print(len(arp_df))
print(len(arp_df.columns))
#arp_df.columns

3487839
282


In [3]:
#len(arp_df.ID) # = arp_df.ID.unique() = 3513617 records from '2016-02-08 00:37:08' till '2020-06-30 23:18:09'
#min(arp_df.Start_Time)
#len(arp_df.Start_Lat.unique())         #1124695
#len(arp_df.Description.unique())       #1780093
#len(arp_df.Number.unique())              #40366
#len(arp_df.Street.unique())             #176262
#len(arp_df.City.unique())                #11896
#len(arp_df.County.unique())               #1724
#len(arp_df.State.unique())                  #49
#len(arp_df.Timezone.unique())                #5
#len(arp_df.Airport_Code.unique())         #2002
#len(arp_df.Weather_Timestamp.unique())  #546087
#len(arp_df.Wind_Direction.unique())         #25
#len(arp_df.Weather_Condition.unique())     #128
#len(arp_df.Amenity.unique()) #,'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 
#           'Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop'  #all True/False   #2
#len(arp_df.Sunrise_Sunset.unique())          #3
#len(arp_df.Civil_Twilight.unique())          #3
#len(arp_df.Nautical_Twilight.unique())       #3

In [4]:
arp_df.Source.unique()

array(['MapQuest', 'MapQuest-Bing', 'Bing'], dtype=object)

In [5]:
#arp_df.TMC.isna()
arp_df.Side.value_counts()

R    2879797
L     633819
           1
Name: Side, dtype: int64

In [6]:
arp_df.Severity.unique()

array([3, 2, 1, 4])

In [7]:
arp_df.Timezone.unique()

array(['US/Eastern', 'US/Pacific', nan, 'US/Central', 'US/Mountain'],
      dtype=object)

In [8]:
arp_df.Amenity.unique()  #boolean column starts

array([0, 1])

In [9]:
arp_df.Turning_Loop.unique()  #boolean column ends

array([0])

In [4]:
#len(arp_df.Weather_Timestamp.unique())
arp_df.Traffic_Signal.head(5)

0    0
1    0
2    1
3    0
4    1
Name: Traffic_Signal, dtype: int64

In [11]:
arp_df.Sunrise_Sunset.unique()

array(['Night', 'Day', nan], dtype=object)

In [12]:
arp_df.Civil_Twilight.unique()

array(['Night', 'Day', nan], dtype=object)

In [13]:
arp_df.Nautical_Twilight.unique()

array(['Night', 'Day', nan], dtype=object)

In [14]:
arp_df.Astronomical_Twilight.unique()

array(['Night', 'Day', nan], dtype=object)

In [15]:
arp_df.Wind_Direction.unique()

array(['CALM', 'SW', 'SSW', 'WSW', 'WNW', 'NW', 'W', 'NNW', 'NNE', 'S',
       'N', 'VAR', 'SE', 'SSE', 'ESE', 'E', 'NE', 'ENE', nan],
      dtype=object)

In [16]:
arp_df['Street'].head(5)

0                       I-70 E
1                     Brice Rd
2               State Route 32
3                       I-75 S
4    Miamisburg Centerville Rd
Name: Street, dtype: object

<em>Wind direction data to be cleaned to have meaningful unique directions </em>

<h2>Feature Extraction and Data Cleaning</h2>

<em> If already ran once, then read pickle file only with command at end of the cell</em>

<h3> Cleaning redundant columns of wind direction </h3>

In [18]:
print('No of columns in dataframe at start '+str(len(arp_df.columns)))   #should be 49
mod1_arp_df = get_time_features(arp_df) #6 columns added: accDuration, day, month, year, dayOfWeek, geometry
mod2_arp_df = convert_boolean_features(mod1_arp_df) #13 columns converted
mod3_arp_df = rename_bracketed_columns(mod2_arp_df) #8 columns renamed
print("Wind Direction before data cleanup: ", arp_df['Wind_Direction'].unique())
arp_df = clean_wind_direction_data(mod3_arp_df)  #clean Wind_Direction data
print("Wind Direction after simplification: ", arp_df['Wind_Direction'].unique())
arp_df.to_pickle('../data_files/US_Accidents_June20_modified.pkl')
print('No of columns in dataframe after all operations '+str(len(arp_df.columns)))  #should be 57
#arp_df = pd.read_pickle('../data_files/US_Accidents_June20_modified.pkl')

No of columns in dataframe at start 49
geo-time features ...
accDuration added to the df in 2 sec
geometry field added to the df in 35 sec
day,month,year,dayOfWeek added to the df in 4 sec
boolean conversion features ...
renaming few weather columns ...
Wind Direction before data cleanup:  ['Calm' 'SW' 'SSW' 'WSW' 'WNW' 'NW' 'West' 'NNW' 'NNE' 'South' 'North'
 'Variable' 'SE' 'SSE' 'ESE' 'East' 'NE' 'ENE' 'E' 'W' nan 'S' 'VAR'
 'CALM' 'N']
cleaning Wind_Direction data ...
Wind Direction after simplification:  ['CALM' 'SW' 'SSW' 'WSW' 'WNW' 'NW' 'W' 'NNW' 'NNE' 'S' 'N' 'VAR' 'SE'
 'SSE' 'ESE' 'E' 'NE' 'ENE' nan]
No of columns in dataframe after all operations 57


<em> Can experiment using data from Aug 2017 until June 2020, i.e. 35 months data </em>
<em> Data before Aug 17 for many states are not fully collected as only one of two api was active </em>

In [21]:
##print(len(arp_df)) #3513617 total records
##flt_arp_df = arp_df.loc[arp_df['Start_Time']>'2017-08-01 00:00:00']
##print(len(flt_arp_df))  ##2759781 records
##arp_df = flt_arp_df

<h2> Work for v1 of dataset starts </h2>

<h3>Columns to be dropped</h3>
<em> Country & Turning_Loop has only one value so not helpful </em> <br>
<em> Wind_Chill_F, Number have more than 55% null values</em> <br>
<em> End_Lat, End_Lng have more than 50% null values, alt. for use is Start_Lat, Start_Lng with distance, still we do not drop them</em>

In [22]:
arp_df = drop_unwanted_columns(arp_df)
print(len(arp_df.columns))
#53 columns remain

53


<h3>Adding geohashes</h3>

<em>Add 3 columns for geohashes which are geohash4, geohash5 and geohash6. geohash5 is area 4.89*4.89 KM sq </em>

In [23]:
geo_arp_df = add_geohashes(arp_df)
print(len(geo_arp_df))
print(len(geo_arp_df.columns))
print(geo_arp_df.head(2))
arp_df = geo_arp_df

adding geohashes...
geometry, geohash4 to geohash6 added to the df in 1605 sec
3513617
56
    ID    Source    TMC  Severity           Start_Time             End_Time  \
0  A-1  MapQuest  201.0         3  2016-02-08 05:46:00  2016-02-08 11:00:00   
1  A-2  MapQuest  201.0         2  2016-02-08 06:07:59  2016-02-08 06:37:59   

   Start_Lat  Start_Lng  End_Lat  End_Lng  ...             endTime  \
0  39.865147 -84.058723      NaN      NaN  ... 2016-02-08 11:00:00   
1  39.928059 -82.831184      NaN      NaN  ... 2016-02-08 06:37:59   

  accDuration                    geometry day month  year dayOfWeek geohash4  \
0       314.0  POINT (-84.05872 39.86515)   8     2  2016         0     dph4   
1        30.0  POINT (-82.83118 39.92806)   8     2  2016         0     dpj5   

  geohash5 geohash6  
0    dph4z   dph4z1  
1    dpj55   dpj55h  

[2 rows x 56 columns]


In [24]:
arp_df.to_pickle('../data_files/US_Accidents_June20_modified_v1.pkl')

<h3>Distinctive weather conditions grouped in bins to reduce redundancy </h3>

In [3]:
# show distinctive weather conditions 
weather ='!'.join(arp_df['Weather_Condition'].dropna().unique().tolist())
weather = np.unique(np.array(re.split(
    "!|\s/\s|\sand\s|\swith\s|Partly\s|Mostly\s|Blowing\s|Freezing\s", weather))).tolist()
print("Weather Conditions: ", weather)

Weather Conditions:  ['', 'Clear', 'Cloudy', 'Drifting Snow', 'Drizzle', 'Dust', 'Dust Whirls', 'Dust Whirls Nearby', 'Dust Whirlwinds', 'Fair', 'Fog', 'Funnel Cloud', 'Hail', 'Haze', 'Heavy ', 'Heavy Drizzle', 'Heavy Ice Pellets', 'Heavy Rain', 'Heavy Rain Shower', 'Heavy Rain Showers', 'Heavy Sleet', 'Heavy Smoke', 'Heavy Snow', 'Heavy T-Storm', 'Heavy Thunderstorms', 'Ice Pellets', 'Light ', 'Light Drizzle', 'Light Fog', 'Light Hail', 'Light Haze', 'Light Ice Pellets', 'Light Rain', 'Light Rain Shower', 'Light Rain Showers', 'Light Sleet', 'Light Snow', 'Light Snow Grains', 'Light Snow Shower', 'Light Snow Showers', 'Light Thunderstorm', 'Light Thunderstorms', 'Low Drifting Snow', 'Mist', 'N/A Precipitation', 'Overcast', 'Partial Fog', 'Patches of Fog', 'Rain', 'Rain Shower', 'Rain Showers', 'Sand', 'Scattered Clouds', 'Shallow Fog', 'Showers in the Vicinity', 'Sleet', 'Small Hail', 'Smoke', 'Snow', 'Snow Grains', 'Snow Showers', 'Squalls', 'T-Storm', 'Thunder', 'Thunder in the Vici

In [4]:
#function to categorise redundant weather conditions data in separate bins 'clear','cloud','rain','heavyRain',
#'snow','heavySnow','fog' added as columns with vales 0 or 1 and then later drop Weather_Condition
arp_df = categorise_weather_conditions(arp_df)
print(len(arp_df))
print(len(arp_df.columns))
print(arp_df.head(2))

3513617
62
    ID    Source    TMC  Severity           Start_Time             End_Time  \
0  A-1  MapQuest  201.0         3  2016-02-08 05:46:00  2016-02-08 11:00:00   
1  A-2  MapQuest  201.0         2  2016-02-08 06:07:59  2016-02-08 06:37:59   

   Start_Lat  Start_Lng  End_Lat  End_Lng  ...  geohash4 geohash5 geohash6  \
0  39.865147 -84.058723      NaN      NaN  ...      dph4    dph4z   dph4z1   
1  39.928059 -82.831184      NaN      NaN  ...      dpj5    dpj55   dpj55h   

  clear cloud rain heavyRain snow heavySnow fog  
0     0     0    1         0    0         0   0  
1     0     0    1         0    0         0   0  

[2 rows x 62 columns]


<h3> Handling missing values </h3>

<em>Filling NaN values of Precipitation_in with median and adding a new column of precipitationNA with value 1 at these where value of Precipitation_in is NaN</em>

In [5]:
arp_df = fill_values_for_precipitation(arp_df)
#number or rows remain same as before, columns increase by 1
print(len(arp_df))
print(len(arp_df.columns))

3513617
63


<em> Counts of missing values in some features are much smaller compared to the total sample. It is convenient to drop rows with missing values in these columns like City, Zipcode, Airport_Code, Sunrise_Sunset, Civil_Twilight,
Nautical_Twilight, Astronomical_Twilight <em>

In [6]:
arp_df = drop_rows_wit_col_1pct_NaN(arp_df)
print(len(arp_df))
print(len(arp_df.columns))

3506744
63


<em> Filling in missing values of some attributes(like Temperature_F, Humidity_pct, Pressure_in, Visibility_mi, 
Wind_Speed_mph) with their mean values </em>

In [8]:
arp_df = fill_values_with_median(arp_df)
print(len(arp_df))
print(len(arp_df.columns))

The number of remaining missing values: 
Temperature_F : 7134
Humidity_pct : 7143
Pressure_in : 7106
Visibility_mi : 17548
Wind_Speed_mph : 17590
3488606
63


<em>For categorical weather features, majority rather than median will be used to replace missing values. </em>

In [9]:
print(arp_df['rain'].isnull().values.any())
arp_df.head(5)

True


,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,geohash5,geohash6,clear,cloud,rain,heavyRain,snow,heavySnow,fog,precipitationNA
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,dph4z,dph4z1,0,0,1,0,0,0,0,0
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,dpj55,dpj55h,0,0,1,0,0,0,0,0
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,dnunp,dnunpz,0,1,0,0,0,0,0,1
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,dph45,dph45g,0,1,0,0,0,0,0,1
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,dph1k,dph1kn,0,1,0,0,0,0,0,1


<em> Filling in some values with majority and then dropping a few </em>

In [10]:
arp_df = fill_values_with_majority(arp_df)
print(len(arp_df))
print(len(arp_df.columns))

Count of missing values that will be dropped: 
Wind_Direction : 609
clear : 628
cloud : 657
rain : 628
heavyRain : 620
snow : 633
heavySnow : 620
fog : 620
3487860
63


<em> Following code helps join a larger df to a smaller df with final size of smaller df</em>

In [11]:
#bigger_df = pd.read_pickle('../data_files/US_Accidents_June20_modified.pkl')
#print(len(bigger_df))
#print(len(arp_df))
#sel_df = bigger_df[['ID','Precipitation_in']]
#sel_df = sel_df[sel_df.ID.isin(list(arp_df['ID']))]
#print(len(sel_df))
#new_arp_df = arp_df.join(sel_df.set_index('ID'), on='ID')
print(len(arp_df.columns))
print(arp_df['rain'].isnull().values.any())

False


<h3> Adding boolean columns for Severity </h3>

In [12]:
#add boolean columns for Severity
print(len(arp_df.columns))
print(len(arp_df))
arp_df = add_boolean_severity(arp_df)
print(len(arp_df.columns))
print(len(arp_df))

63
3487860
4 columns with boolean values added for severity 1,2,3,4
67
3487860


<em> 4 columns added(severity1, severity2, severity3, severity4) </em>

<h3> Adding boolean columns for Timezones </h3>

In [13]:
#add boolean columns for Timezones
print(len(arp_df.columns))
print(len(arp_df))
arp_df = add_boolean_timezone(arp_df)
print(len(arp_df.columns))
print(len(arp_df))

67
3487860
4 columns with boolean values added for timezones US/Eastern, US/Pacific, US/Central, US/Mountain 
71
3487860


<em> 4 columns added(US/Eastern, US/Pacific, US/Central, US/Mountain) </em>

In [14]:
#sanity checks
print(sum(arp_df['timezoneUS/Eastern']))
print(sum(arp_df['timezoneUS/Pacific']))
print(sum(arp_df['timezoneUS/Central']))
print(sum(arp_df['timezoneUS/Mountain']))

1471090
980170
836472
200128


<h3> Adding boolean columns for Roadside </h3>

In [15]:
#add boolean columns for Roadside
print(len(arp_df.columns))
print(len(arp_df))
arp_df = add_boolean_roadside(arp_df)
print(len(arp_df.columns))
print(len(arp_df))

71
3487860
2 columns with boolean values R/L for side of road added
73
3487859


<em> 2 columns for Road Side added R/L</em>

In [16]:
#sanity checks
print(sum(arp_df['roadsideR']))
print(sum(arp_df['roadsideL']))

2857883
629976


<h3> Adding boolean columns for Day/Night </h3>

In [17]:
#add boolean columns for Day/Night of Sunrise_Sunset','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight
print(len(arp_df.columns))
print(len(arp_df))
arp_df = add_boolean_day_night(arp_df)
print(len(arp_df.columns))
print(len(arp_df))

73
3487859
8 columns with boolean values Day/Night added
81
3487859


<em> 8 columns with boolean values Day/Night added </em>

In [18]:
#sanity checks
print(sum(arp_df['timeSSDay']))
print(sum(arp_df['timeSSNight']))
print(sum(arp_df['timeATDay']))
print(sum(arp_df['timeATNight']))
print(sum(arp_df['timeCTDay']))
print(sum(arp_df['timeCTNight']))
print(sum(arp_df['timeNTDay']))
print(sum(arp_df['timeNTNight']))

2575281
912578
3053166
434693
2748270
739589
2922543
565316


<h3> Adding boolean columns for Wind Direction </h3>

In [19]:
#add boolean columns for Wind Direction
print(len(arp_df.columns))
print(len(arp_df))
arp_df = add_boolean_wind_direction(arp_df)
print(len(arp_df.columns))
print(len(arp_df))

81
3487859
18 columns with boolean values added for wind direction
99
3487859


In [20]:
print(sum(arp_df['windDirectionCALM']))
print(sum(arp_df['windDirectionSW']))
print(sum(arp_df['windDirectionSSW']))
print(sum(arp_df['windDirectionWSW']))
print(sum(arp_df['windDirectionWNW']))
print(sum(arp_df['windDirectionNW']))
print(sum(arp_df['windDirectionW']))
print(sum(arp_df['windDirectionNNW']))
print(sum(arp_df['windDirectionNNE']))
print(sum(arp_df['windDirectionS']))
print(sum(arp_df['windDirectionN']))
print(sum(arp_df['windDirectionVAR']))
print(sum(arp_df['windDirectionSSE']))
print(sum(arp_df['windDirectionESE']))
print(sum(arp_df['windDirectionE']))
print(sum(arp_df['windDirectionNE']))
print(sum(arp_df['windDirectionENE']))

606110
173280
182130
166291
174532
165621
262934
147458
117713
283516
221463
181110
164776
114913
164122
116428
112836


<em> 18 columns with boolean values added for wind direction </em>

<h3>Adding bins with boolean values for TMC values </h3>

In [21]:
print(len(arp_df.columns))
print(len(arp_df))
arp_df = add_boolean_TMC(arp_df)
print(len(arp_df.columns))
print(len(arp_df))

99
3487859
22 columns with boolean values added for TMC
121
3487859


In [22]:
#print(sum(arp_df['tmc15']))
#print(arp_df.tmc21.sum())
#arp_df.tmc0.head(2)
#arp_df.columns
arp_df['tmc201'].sum()

2068603

<em> Boolean TMC columns added </em>

<h3>Adding bins with boolean values for weather parameters </h3>

In [23]:
print(len(arp_df.columns))
print(len(arp_df))
arp_df = add_bins_temp_humd_presr_visb_winSpeed(arp_df)
print(len(arp_df.columns))
print(len(arp_df))

121
3487859
13 columns with boolean values added for Temperature_F
5 columns with boolean values added for Humidity_pct
4 columns with boolean values added for Pressure_in
7 columns with boolean values added for Visibility_mi
10 columns with boolean values added for Wind_Speed_mph
160
3487839


<em> 51 columns added in total </em>

In [24]:
#print(arp_df.temperature_f2.sum())
#print(arp_df.pressure_in6.sum())  #4
#print(arp_df.visibility_mi2.sum())
#print(arp_df.wind_speed_mph6.sum())  #10
#print(arp_df.pressure_in1.sum())
print(arp_df.humidity_pct1.sum())
max(arp_df['Pressure_in'])

119231


33.04

In [25]:
arp_df.to_pickle('../data_files/US_Accidents_June20_modified_v2.pkl')

<h2> Work for v3 of dataset starts </h2>

<h3> Adding street types </h3>

In [26]:
#add boolean columns for street types where more accidents happen
print(len(arp_df.columns))
print(len(arp_df))
arp_df = streets_high_sev_acc(arp_df)
print(len(arp_df.columns))
print(len(arp_df))
arp_df.head(5)

160
3487839
Most common words in street names are...
Rd, , Dr, St, Ave, N, S, E, W, Ln, Blvd, Highway, Way, Ct, State, Hwy, NE, Pkwy, SW, Pl, SE, NW, Road, Old, Cir, Route, US, Creek, Hill, Park, County, Lake, Trl, Valley, Ridge, Mill, River, Pike, Oak, Ter


Removing some irrelevant names and then adding following 25 columns with boolean values...
 Rd,  St,  Dr,  Ave,  Blvd,  Ln,  Highway,  Pkwy,  Hwy,  Way,  Ct, Pl,  Road, US-, Creek,  Cir, Hill, Route, I-, Trl, Valley, Ridge, Pike,  Fwy, River
185
3487839


,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,cir,hill,route,i-,trl,valley,ridge,pike,fwy,river
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,0,0,1,0,0,0,0,0,0,0
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


<em> 25 columns with street types added </em>

<h3>Add time bins</h3>

In [27]:
arp_df = add_time_bins(arp_df)
print(len(arp_df.columns))

282


<em> 1(timeQuarterOfDay)+96(quarter1,quarter2,...,quarter96) new columns added</em>

In [28]:
arp_df.to_pickle('../data_files/US_Accidents_June20_modified_v3.pkl')

<h3> Dealing with Descriptions</h3>

In [5]:
arp_df[['Description','Severity']]

,Description,Severity
0,Right lane blocked due to accident on I-70 Eas...,3
1,Accident on Brice Rd at Tussing Rd. Expect del...,2
2,Accident on OH-32 State Route 32 Westbound at ...,2
3,Accident on I-75 Southbound at Exits 52 52B US...,3
4,Accident on McEwen Rd at OH-725 Miamisburg Cen...,2
...,...,...
3513612,At Market St - Accident.,2
3513613,At Camino Del Rio/Mission Center Rd - Accident.,2
3513614,At Glassell St/Grand Ave - Accident. in the ri...,2
3513615,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,2


In [9]:
#Checking where value of the description column is NaN
arp_df.loc[arp_df['Description'].isna()]

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Cir,Hill,Route,I-,Trl,Valley,Ridge,Pike,Fwy,River
1295828,A-1295857,MapQuest,201.0,3,2019-03-20 17:20:14,2019-03-20 18:48:38,29.776676,-95.38855,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#filling one of the description column with value as 'Accident'
arp_df.at[1295828,'Description']= 'Accident.'
arp_df.loc[arp_df['ID']=='A-1295857']['Description']
arp_df['Description'].isna().sum()

0

In [25]:
lst_desc = list(arp_df['Description'])
print(len(lst_desc))

3487860


In [43]:
#traffic events: accident,broken vehicle, congestion, construction, event, lane-blocked, and flow incident
cnt_cons = 0
cnt_block = 0
cnt_cong = 0
cnt_broke = 0
cnt_event = 0
cnt_flow = 0
cnt_acc = 0
cnt_other = 0
for sent in lst_desc:
    if 'construction' in sent.lower():
        cnt_cons+=1
    if 'lane blocked' in sent.lower() or 'blocked'  in sent.lower()  or 'closed'  in sent.lower() or 'restrictions' in sent.lower():
        cnt_block += 1
    if 'congestion' in sent.lower():
        cnt_cong += 1
    if 'spillage' in sent.lower() or 'jackknifed' in sent.lower() or 'struck' in sent.lower() or 'overturned' in sent.lower()  or 'spun' in sent.lower():
        cnt_broke += 1
    if 'event' in sent.lower():
        cnt_event += 1
    if 'flow' in sent.lower():
        cnt_flow += 1
    if 'accident' in sent.lower():
        cnt_acc += 1
print(cnt_cons,cnt_block,cnt_cong,cnt_broke,cnt_event,cnt_flow,cnt_acc,cnt_other)

340 1182179 0 11552 169 2527 2291093 0


In [24]:
arp_df['Description'].tail(20)

3513597        At Friars Rd - Accident. Center lane blocked.
3513598                        At CA-14/Exit 162 - Accident.
3513599       At CA-2/Santa Monica Blvd/Exit 55A - Accident.
3513600                     At Home Ave/Exit 13B - Accident.
3513601                              At Adams St - Accident.
3513602                         At Archibald Ave - Accident.
3513603    Ramp closed to The City Dr/Exit 14A - Road clo...
3513604              At CA-118/Ronald Reagan Fwy - Accident.
3513605                            At Tustin Ave - Accident.
3513606                          At CA-210/CA-57 - Accident.
3513607                                 At I-605 - Accident.
3513608                       At Jack Rabbit Trl - Accident.
3513609                  At Telephone Rd/Exit 65 - Accident.
3513610                                 At CA-55 - Accident.
3513611                   At Osborne St/Exit 154 - Accident.
3513612                             At Market St - Accident.
3513613      At Camino D

In [28]:
print(len(arp_df))

3487860


<em> Adding description is not of much help as more than 90% text has accident mentioned </em>

<h2> 25th Nov: Preparing train test files for Models </h2>

<h3> on v3 Dataset having 228 columns obtained after applying feature engineering </h3>

In [3]:
arp_v3_df = pd.read_pickle('../data_files/US_Accidents_June20_modified_v3.pkl')
arp_v3_df.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng',
       ...
       'quarter87', 'quarter88', 'quarter89', 'quarter90', 'quarter91',
       'quarter92', 'quarter93', 'quarter94', 'quarter95', 'quarter96'],
      dtype='object', length=282)

In [5]:
list(arp_df.columns)
#arp_df.geohash5.head(5)

['ID',
 'Source',
 'TMC',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance_mi',
 'Description',
 'Street',
 'Side',
 'City',
 'County',
 'State',
 'Zipcode',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature_F',
 'Humidity_pct',
 'Pressure_in',
 'Visibility_mi',
 'Wind_Direction',
 'Wind_Speed_mph',
 'Precipitation_in',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight',
 'startTime',
 'endTime',
 'accDuration',
 'geometry',
 'day',
 'month',
 'year',
 'dayOfWeek',
 'geohash4',
 'geohash5',
 'geohash6',
 'clear',
 'cloud',
 'rain',
 'heavyRain',
 'snow',
 'heavySnow',
 'fog',
 'precipitationNA',
 'severity1',
 'severity2',
 'severity3',
 'severity4',
 'timezoneUS/Eastern',
 'timezoneUS/Pacific',
 'timezoneUS/Central',
 'timezo

In [4]:
arp_df.loc[arp_df['City']=='Atlanta']['geohash5'].value_counts() 

dn5bp    6064
djgzz    4635
dn5bj    2852
dnh02    2619
dn5br    2407
         ... 
dr8fj       1
dp0sc       1
dp4uf       1
dp4ub       1
dp4uw       1
Name: geohash5, Length: 67, dtype: int64

In [14]:
lst_test_cities_geohashes = ['dqche', 'c23n8', 'dn5bp', '9tbq2']

In [17]:
tt_ssap_g5_df = arp_df.loc[arp_df['geohash5'].isin(lst_test_cities_geohashes)]
tt_ssap_g5_df

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,quarter87,quarter88,quarter89,quarter90,quarter91,quarter92,quarter93,quarter94,quarter95,quarter96
146266,A-146268,MapQuest,229.0,3,2016-11-30 15:49:32,2016-11-30 17:20:00,33.791683,-84.391472,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
146267,A-146269,MapQuest,201.0,2,2016-11-30 15:48:26,2016-11-30 17:18:08,33.751411,-84.390968,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
146268,A-146270,MapQuest,229.0,3,2016-11-30 15:11:50,2016-11-30 16:56:31,33.781532,-84.390869,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
146269,A-146271,MapQuest,229.0,3,2016-11-30 14:41:44,2016-11-30 16:56:18,33.771290,-84.390472,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
146297,A-146299,MapQuest,229.0,3,2016-11-30 19:00:35,2016-11-30 19:29:44,33.769001,-84.390511,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3513005,A-3513169,Bing,NaN,3,2019-08-22 07:03:27,2019-08-22 07:30:10,47.557591,-122.320461,47.558781,-122.321050,...,0,0,0,0,0,0,0,0,0,0
3513045,A-3513209,Bing,NaN,2,2019-08-22 12:26:04,2019-08-22 12:54:22,47.560300,-122.339380,47.560300,-122.339380,...,0,0,0,0,0,0,0,0,0,0
3513420,A-3513584,Bing,NaN,2,2019-08-23 16:45:28,2019-08-23 17:15:00,33.468636,-112.111228,33.468636,-112.111228,...,0,0,0,0,0,0,0,0,0,0
3513456,A-3513620,Bing,NaN,2,2019-08-23 09:49:57,2019-08-23 10:19:30,47.562573,-122.321289,47.564800,-122.320860,...,0,0,0,0,0,0,0,0,0,0


In [18]:
tt_ssap_g5_df.City.value_counts()

Atlanta        6064
Seattle        3557
Phoenix        2026
Springfield    1196
Alexandria       12
Name: City, dtype: int64

In [19]:
tt_ssap_g5_df.to_pickle('../data_files/tt_ssap_g5.pkl')

In [9]:
#arp_df['Distance_mi'].isna().any()
print(min(arp_df['Distance_mi']))
print(max(arp_df['Distance_mi']))

0.0
333.63000488299997


In [9]:
#X_test_Atlanta = np.load('../data_files/dataset/X_test_Atlanta.npy',allow_pickle=True)
print(X_test_Atlanta.shape)
X_test_Atlanta

(2515, 315)


array([[1, 0, 0, ..., 0.518022355827172, -0.06623753526029841, 98],
       [1, 0, 0, ..., 0.518022355827172, -0.06623753526029841, 98],
       [1, 0, 1, ..., 0.518022355827172, -0.06623753526029841, 98],
       ...,
       [1, 0, 1, ..., 0.6725878851606645, -0.025870411158965507, 489],
       [1, 0, 1, ..., 0.6725878851606645, -0.025870411158965507, 489],
       [1, 0, 1, ..., 0.6725878851606645, -0.025870411158965507, 489]],
      dtype=object)

In [10]:
X_train_Atlanta = np.load('../data_files/dataset/X_train_Atlanta.npy',allow_pickle=True)
print(X_train_Atlanta.shape)
X_train_Atlanta

(12085, 315)


array([[1, 0, 0, ..., 0.518022355827172, -0.06623753526029841, 98],
       [1, 0, 1, ..., 0.518022355827172, -0.06623753526029841, 98],
       [0, 0, 1, ..., 0.518022355827172, -0.06623753526029841, 98],
       ...,
       [1, 0, 1, ..., 0.6725878851606645, -0.025870411158965507, 489],
       [1, 0, 1, ..., 0.6725878851606645, -0.025870411158965507, 489],
       [1, 0, 1, ..., 0.6725878851606645, -0.025870411158965507, 489]],
      dtype=object)

In [11]:
y_test_Atlanta = np.load('../data_files/dataset/y_test_Atlanta.npy',allow_pickle=True)
print(y_test_Atlanta.shape)
y_test_Atlanta

(2515,)


array([0, 0, 0, ..., 1, 0, 0])

In [12]:
y_train_Atlanta = np.load('../data_files/dataset/y_train_Atlanta.npy',allow_pickle=True)
print(y_train_Atlanta.shape)
y_train_Atlanta

(12085,)


array([0, 0, 0, ..., 0, 0, 0])

In [34]:
#print(list(arp_df.columns))
arp_df.day.tail(5)

3513612    23
3513613    23
3513614    23
3513615    23
3513616    23
Name: day, dtype: int64

In [28]:
cities = ['Houston', 'Atlanta', 'Phoenix', 'Seattle']
cities_df = arp_df.loc[arp_df['City'].isin(cities)]
print(len(cities_df.geohash5.unique()))  
print(len(cities_df))
#Houston 147, 101240
#Atlanta  67,  41126
#Phoenix  93,  28553
#Seattle  35,  26056

In [3]:
arp_df.head(2)

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,quarter87,quarter88,quarter89,quarter90,quarter91,quarter92,quarter93,quarter94,quarter95,quarter96
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [3]:
list(arp_df.columns)

['ID',
 'Source',
 'TMC',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance_mi',
 'Description',
 'Street',
 'Side',
 'City',
 'County',
 'State',
 'Zipcode',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature_F',
 'Humidity_pct',
 'Pressure_in',
 'Visibility_mi',
 'Wind_Direction',
 'Wind_Speed_mph',
 'Precipitation_in',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight',
 'startTime',
 'endTime',
 'accDuration',
 'geometry',
 'day',
 'month',
 'year',
 'dayOfWeek',
 'geohash4',
 'geohash5',
 'geohash6',
 'clear',
 'cloud',
 'rain',
 'heavyRain',
 'snow',
 'heavySnow',
 'fog',
 'precipitationNA',
 'severity1',
 'severity2',
 'severity3',
 'severity4',
 'timezoneUS/Eastern',
 'timezoneUS/Pacific',
 'timezoneUS/Central',
 'timezo

<h3> Preparing another train and test data using sub-sampling </h3>

In [57]:
#Going by geohash5
#Springfield = ['dqchf','dqchg','dqchu','dqchd','dqche','dqchs','dqch6','dqch7','dqchk'] #dqch
#Seattle = ['c22yz','c23nb','c23nc','c22yx','c23n8','c23n9','c22yr','c23n2','c23n3'] #c23n
#Atlanta = ['dn5bq','dn5br','dnh02','dn5bn','dn5bp','dnh00','djgzy','djgzz','djupb'] #dn5b
#Phoenix = ['9tbnx','9tbq8','9tbq9','9tbnr','9tbq2','9tbq3','9tbnp','9tbq0','9tbq1'] #9tbq
#Going by geohash4
#Los_Angeles = ['9q5c'] #82.3k
#Houston = ['9vk1'] #57.8k
#Seattle = ['c23n'] #20.3k
#Phoenix = ['9tbq'] #20.2k
#Atlanta = atlanta_df.geohash5.unique()
#lst_LHSRP = Los_Angeles + Houston + Seattle + Raleigh + Phoenix
#g4_LHSRP_df = arp_df.loc[arp_df['geohash4'].isin(lst_LHSRP)]
#print(len(g4_LHSRP_df)) #5 different cities from different part of USA, #220020 records
#print(len(g4_LHSRP_df.geohash5.unique())) #total 159 areas of 4.89*4.89 sq KM for which training and prediction
#ssap_df = arp_df.loc[arp_df['geohash4'].isin(lst_ssap)]
#print(len(ssap_df))  #69286
#Houston 147, 101240
#Atlanta  67,  41126
#Phoenix  93,  28553
#Seattle  35,  26056

In [5]:
lst_columns_ss = ['ID','month','year','geohash5','Side','City','dayOfWeek','Amenity','Bump','Crossing',  
'Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal',
'clear','cloud','rain','heavyRain','snow','heavySnow','fog',
'timezoneUS/Eastern','timezoneUS/Pacific','timezoneUS/Central','timezoneUS/Mountain','timeSSDay',
'windDirectionCALM','windDirectionSW','windDirectionSSW','windDirectionWSW','windDirectionWNW','windDirectionNW',
'windDirectionW','windDirectionNNW','windDirectionNNE','windDirectionS','windDirectionN','windDirectionVAR',
'windDirectionSE','windDirectionSSE','windDirectionESE','windDirectionE','windDirectionNE','windDirectionENE',
'temperature_f1','temperature_f2','temperature_f3','temperature_f4','temperature_f5',
'temperature_f6','temperature_f7','temperature_f8','temperature_f9','temperature_f10','temperature_f11',
'temperature_f12','temperature_f13','humidity_pct1','humidity_pct2','humidity_pct3','humidity_pct4',
'humidity_pct5','pressure_in1','pressure_in2','pressure_in3','pressure_in4','visibility_mi1','visibility_mi2',
'visibility_mi3','visibility_mi4','visibility_mi5','visibility_mi6','visibility_mi7','wind_speed_mph1',
'wind_speed_mph2','wind_speed_mph3','wind_speed_mph4','wind_speed_mph5','wind_speed_mph6','wind_speed_mph7',
'wind_speed_mph8','wind_speed_mph9','wind_speed_mph10','rd','st','dr','ave','blvd','ln','highway','pkwy','hwy',
'way','ct','pl','road','us-','creek','cir','hill','route','i-','trl','valley','ridge','pike','fwy','river',
'quarter33','quarter34','quarter35','quarter36','quarter37',
'quarter38','quarter39','quarter40','quarter41']

In [11]:
#arp_df = arp_df.loc[arp_df['City']=='Atlanta']
#arp_df = add_boolean_geohash(arp_df)
cities = ['Houston', 'Atlanta', 'LosAngeles', 'Charlotte', 'Dallas', 'Austin']
cities_df = arp_df.loc[arp_df['City'].isin(cities)]
cities_df = add_boolean_geohash(cities_df)
print('Total unique geohashes as contained by cities',len(cities_df.geohash5.unique()))  
print('Total sample size of cases of the above cities',len(cities_df))
print(len(cities_df.columns))

540 columns with boolean values added for geohash5
Total unique geohashes as contained by cities 540
Total sample size of cases of the above cities 349959
823


<em> For all cities for its geohash5, an equivalent 282 sized one hot vector gets added </em>

In [12]:
#for taking a sample of data from one season, we take data from June to august first then later March to May
#also, taking only those columns which we need to use for train-test, train on time 8-10, predict on time 10-10:15
#i.e. quarter33 to quarter41
#processing only fro Atlanta
arp_ss_train_df = cities_df.query('(month==6 | month ==7) | (month==8 & day<18)')
arp_ss_test_df = cities_df.query('month==8 & day>=18')
#arp_ss_train_df = arp_ss_train_df[lst_columns_ss]
#arp_ss_test_df = arp_ss_test_df[lst_columns_ss]
print(len(arp_ss_train_df.columns))  #122
print(len(arp_ss_train_df)) #684913
print(len(arp_ss_test_df))  #131101

823
70094
14440


In [13]:
#column weekday or not added and later we remove dayOfWeek
arp_ss_train_df = get_WeekdayWeekend(arp_ss_train_df)
print(len(arp_ss_train_df.columns))
print(len(arp_ss_train_df))

weekday column added having boolean values
824
70094


In [14]:
#column weekday or not added and later we remove dayOfWeek
arp_ss_test_df = get_WeekdayWeekend(arp_ss_test_df)
print(len(arp_ss_test_df.columns))
print(len(arp_ss_test_df))

weekday column added having boolean values
824
14440


In [15]:
lst_columns_ss = ['ID','month','year','geohash5','Side','City','dayOfWeek','Amenity','Bump','Crossing',  
'Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal',
'clear','cloud','rain','heavyRain','snow','heavySnow','fog',
'timezoneUS/Eastern','timezoneUS/Pacific','timezoneUS/Central','timezoneUS/Mountain','timeSSDay','timeSSNight',
'windDirectionCALM','windDirectionSW','windDirectionSSW','windDirectionWSW','windDirectionWNW','windDirectionNW',
'windDirectionW','windDirectionNNW','windDirectionNNE','windDirectionS','windDirectionN','windDirectionVAR',
'windDirectionSE','windDirectionSSE','windDirectionESE','windDirectionE','windDirectionNE','windDirectionENE',
'temperature_f1','temperature_f2','temperature_f3','temperature_f4','temperature_f5',
'temperature_f6','temperature_f7','temperature_f8','temperature_f9','temperature_f10','temperature_f11',
'temperature_f12','temperature_f13','humidity_pct1','humidity_pct2','humidity_pct3','humidity_pct4',
'humidity_pct5','pressure_in1','pressure_in2','pressure_in3','pressure_in4','visibility_mi1','visibility_mi2',
'visibility_mi3','visibility_mi4','visibility_mi5','visibility_mi6','visibility_mi7','wind_speed_mph1',
'wind_speed_mph2','wind_speed_mph3','wind_speed_mph4','wind_speed_mph5','wind_speed_mph6','wind_speed_mph7',
'wind_speed_mph8','wind_speed_mph9','wind_speed_mph10','rd','st','dr','ave','blvd','ln','highway','pkwy','hwy',
'way','ct','pl','road','us-','creek','cir','hill','route','i-','trl','valley','ridge','pike','fwy','river',
'quarter33','quarter34','quarter35','quarter36','quarter37',
'quarter38','quarter39','quarter40','quarter41']

In [17]:
g5_col = ['geohash5']
roadSideRL_col = ['roadsideRL']
city_side_col = ['City','Side']
other_cols = ['Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway',
'Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','clear','cloud','rain','heavyRain','snow',
'heavySnow','fog','timezoneUS/Eastern','timezoneUS/Pacific','timezoneUS/Central',
'timezoneUS/Mountain','weekday','timeSSDay',
'windDirectionCALM','windDirectionSW','windDirectionSSW','windDirectionWSW',
'windDirectionWNW','windDirectionNW','windDirectionW','windDirectionNNW','windDirectionNNE','windDirectionS',
'windDirectionN','windDirectionVAR','windDirectionSE','windDirectionSSE','windDirectionESE','windDirectionE',
'windDirectionNE','windDirectionENE',
'temperature_f1','temperature_f2','temperature_f3','temperature_f4','temperature_f5',
'temperature_f6','temperature_f7','temperature_f8','temperature_f9','temperature_f10','temperature_f11',
'temperature_f12','temperature_f13','humidity_pct1','humidity_pct2','humidity_pct3','humidity_pct4',
'humidity_pct5','pressure_in1','pressure_in2','pressure_in3','pressure_in4','visibility_mi1','visibility_mi2',
'visibility_mi3','visibility_mi4','visibility_mi5','visibility_mi6','visibility_mi7','wind_speed_mph1',
'wind_speed_mph2','wind_speed_mph3','wind_speed_mph4','wind_speed_mph5','wind_speed_mph6','wind_speed_mph7',
'wind_speed_mph8','wind_speed_mph9','wind_speed_mph10','rd','st','dr','ave','blvd','ln','highway','pkwy','hwy',
'way','pl','road','us-','creek','cir','hill','route','i-','trl','valley','ridge','pike','fwy','river',
'quarter33','quarter34','quarter35','quarter36','quarter37','quarter38','quarter39','quarter40','quarter41']
g_cols = ['g_'+str(i+1) for i in range(540)] #if using one hot vector equivalent to the gepohashes for 6 cities
emb_cols = ['ge'+str(i+1) for i in range(100)]  #if using geohash embedding of size 100

<em> Using one hot vector equivalent to the gepohashes for 6 cities </em>

In [18]:
tt_cols = g_cols + roadSideRL_col + other_cols
arp_ss_train_df = arp_ss_train_df[tt_cols]
arp_ss_test_df = arp_ss_test_df[tt_cols]

In [19]:
print(len(arp_ss_train_df.columns))
print(len(arp_ss_test_df.columns))

656
656


In [20]:
arp_ss_train_df.to_csv('../data_files/train_set_6cities.csv',index=False)
arp_ss_test_df.to_csv('../data_files/test_set_6cities.csv',index=False)

In [27]:
arp_ss_test_df.head(1)

,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9,g_10,...,river,quarter33,quarter34,quarter35,quarter36,quarter37,quarter38,quarter39,quarter40,quarter41
156248,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
arp_ss_test_df.head(1)

,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9,g_10,...,river,quarter33,quarter34,quarter35,quarter36,quarter37,quarter38,quarter39,quarter40,quarter41
156248,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<em> Using geohash embedding of size 100 for the cities, preparing training and test sets </em>

In [20]:
cities = ['Houston', 'Atlanta', 'LosAngeles', 'Charlotte', 'Dallas', 'Austin']
cities_train_df = arp_ss_train_df.loc[arp_ss_train_df['City'].isin(cities)]
print('Total unique geohashes as contained by cities',len(cities_train_df.geohash5.unique()))  
print('Total sample size of cases of the above cities',len(cities_train_df))
print(len(cities_train_df.columns))

Total unique geohashes as contained by cities 57
Total sample size of cases of the above cities 8580
123


In [21]:
#cities = ['Houston', 'Atlanta', 'LosAngeles', 'Charlotte', 'Dallas', 'Austin']
cities_test_df = arp_ss_test_df.loc[arp_ss_test_df['City'].isin(cities)]
print('Total unique geohashes as contained by cities',len(cities_test_df.geohash5.unique()))  
print('Total sample size of cases of the above cities',len(cities_test_df))
print(len(cities_test_df.columns))

Total unique geohashes as contained by cities 49
Total sample size of cases of the above cities 1897
283


In [22]:
selected_cols_df = cities_train_df[g5_col+city_side_col+other_cols]
selected_cols_df.head(1)
#print(selected_cols_df.head(1))
#print(selected_cols_df.columns[selected_cols_df.isna().any()].tolist())  #should contain nothing if no value in df is NaN

,geohash5,City,Side,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,...,river,quarter33,quarter34,quarter35,quarter36,quarter37,quarter38,quarter39,quarter40,quarter41
155413,djgzm,Atlanta,R,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
selected_cols_test_df = cities_test_df[g5_col+city_side_col+other_cols]
#print(selected_cols_df.head(1))
#print(selected_cols_df.columns[selected_cols_df.isna().any()].tolist())  #should contain nothing if no value in df is NaN

In [24]:
emb_df = embedd_ct_g5(selected_cols_df)
tt_cols = g5_col + city_side_col + emb_cols + roadSideRL_col + other_cols
tt_df = emb_df[tt_cols]
print(len(tt_df.geohash5.unique()))
print(len(tt_df))
print(len(tt_df.columns))
tt_df.head(2)
train_test_cols = emb_cols + roadSideRL_col + other_cols
train_df = tt_df[train_test_cols]
train_df.to_csv('../data_files/train_etc_atlanta_0312.csv',index=False)
#tt_train_df = pd.read_csv('../data_files/train_test_etc_0312.csv')

columns roadsideRL and geohash embedding added
57
8580
219


In [25]:
emb_test_df = embedd_ct_g5(selected_cols_test_df)
tt_cols = g5_col + city_side_col + emb_cols + roadSideRL_col + other_cols
tt_test_df = emb_test_df[tt_cols]
print(len(tt_test_df.geohash5.unique()))
print(len(tt_test_df))
print(len(tt_test_df.columns))
tt_test_df.head(2)
train_test_cols = emb_cols + roadSideRL_col + other_cols
#tt_df = pd.read_csv('../data_files/train_test_etc_0312.csv')
test_df = tt_test_df[train_test_cols]
test_df.to_csv('../data_files/test_etc_atlanta_0312.csv',index=False)

columns roadsideRL and geohash embedding added
49
1897
219


In [28]:
train_df['quarter41'].sum()  #train acc 142, test acc 19, train tot rec 8580, test tot rec 1897

142

<em> <b> 03.12.2020: </b> Preparing dataset by mixing non-accidents cases </em>

In [67]:
train_all_df = pd.read_csv('../data_files/train_etc_0312.csv') 
test_all_df = pd.read_csv('../data_files/test_etc_0312.csv') 
print(len(train_all_df))
print(len(test_all_df))

70094
14440


In [40]:
train_acc_df = train_all_df.query('quarter33==1 | quarter34==1 | quarter35==1 | quarter36==1 \
                                | quarter37==1 | quarter38==1 | quarter39==1 | quarter40==1 | quarter41==1')
print(len(train_acc_df))

12886


In [41]:
test_acc_df = test_all_df.query('quarter33==1 | quarter34==1 | quarter35==1 | quarter36==1 \
                                | quarter37==1 | quarter38==1 | quarter39==1 | quarter40==1 | quarter41==1')
print(len(test_acc_df))

2850


In [53]:
train_acc_df.to_csv('../data_files/train_etc_acc.csv',index=False)
test_acc_df.to_csv('../data_files/test_etc_acc.csv',index=False)

In [52]:
test_acc_df.tail(5)

,ge1,ge2,ge3,ge4,ge5,ge6,ge7,ge8,ge9,ge10,...,river,quarter33,quarter34,quarter35,quarter36,quarter37,quarter38,quarter39,quarter40,quarter41
14413,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,0,0,1,0,0,0,0
14414,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,0,0,1,0,0,0,0
14430,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,1,0,0,0,0,0,0
14432,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,0,0,0,0,0,1,0
14433,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,0,0,0,0,1,0,0


<em> <b> Non-accidents records within 2 hours </b> </em>

In [44]:
train_non_acc_df = train_all_df.query('quarter33==0 & quarter34==0 & quarter35==0 & quarter36==0 \
                                & quarter37==0 & quarter38==0 & quarter39==0 & quarter40==0 & quarter41==0')
print(len(train_non_acc_df))

57208


In [45]:
test_non_acc_df = test_all_df.query('quarter33==0 & quarter34==0 & quarter35==0 & quarter36==0 \
                                & quarter37==0 & quarter38==0 & quarter39==0 & quarter40==0 & quarter41==0')
print(len(test_non_acc_df))

11590


<em> Taking only that amount(23%) of non-acc records which makes total of 50% accidents and 50% non accidents records </em>

In [46]:
train_non_acc_df = train_non_acc_df.sample(frac = 0.23)
print(len(train_non_acc_df))
test_non_acc_df = test_non_acc_df.sample(frac = 0.23)
print(len(test_non_acc_df))

13158
2666


In [48]:
train_all_df = pd.concat([train_acc_df,train_non_acc_df])
print(len(train_all_df))
test_all_df = pd.concat([test_acc_df,test_non_acc_df])
print(len(test_all_df))

26044
5516


In [50]:
train_all_df = train_all_df.sort_index()
test_all_df = test_all_df.sort_index()

In [54]:
train_all_df.to_csv('../data_files/train_etc_accNonacc.csv',index=False)
test_all_df.to_csv('../data_files/test_etc_accNonacc.csv',index=False)
test_all_df.tail(5)

,ge1,ge2,ge3,ge4,ge5,ge6,ge7,ge8,ge9,ge10,...,river,quarter33,quarter34,quarter35,quarter36,quarter37,quarter38,quarter39,quarter40,quarter41
14425,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,0,0,0,0,0,0,0
14430,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,1,0,0,0,0,0,0
14432,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,0,0,0,0,0,1,0
14433,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,0,0,0,0,1,0,0
14439,-0.133037,-0.102192,0.167332,-0.001134,-0.037979,-0.181857,0.012094,0.53773,-0.140604,-0.172879,...,0,0,0,0,0,0,0,0,0,0


<em> Street type wise data preparation for prediction </em>

In [59]:
list(arp_df.columns)

['ID',
 'Source',
 'TMC',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance_mi',
 'Description',
 'Street',
 'Side',
 'City',
 'County',
 'State',
 'Zipcode',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature_F',
 'Humidity_pct',
 'Pressure_in',
 'Visibility_mi',
 'Wind_Direction',
 'Wind_Speed_mph',
 'Precipitation_in',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight',
 'startTime',
 'endTime',
 'accDuration',
 'geometry',
 'day',
 'month',
 'year',
 'dayOfWeek',
 'geohash4',
 'geohash5',
 'geohash6',
 'clear',
 'cloud',
 'rain',
 'heavyRain',
 'snow',
 'heavySnow',
 'fog',
 'precipitationNA',
 'severity1',
 'severity2',
 'severity3',
 'severity4',
 'timezoneUS/Eastern',
 'timezoneUS/Pacific',
 'timezoneUS/Central',
 'timezo

In [61]:
lst_streets = ['rd','st','dr','ave','blvd','ln','highway','pkwy','hwy','way','ct','pl','road','us-','creek','cir','hill',
 'route','i-','trl','valley','ridge','pike','fwy','river']

In [62]:
st_df = arp_df[lst_streets]

In [63]:
st_df.sum()

rd         549856
st         386987
dr         146532
ave        293141
blvd       172042
ln          38812
highway     97554
pkwy        81142
hwy        108564
way         18236
ct           6491
pl          19528
road        16252
us-        110958
creek       15369
cir          5674
hill        20407
route       26542
i-         814371
trl         10589
valley      17885
ridge        8910
pike        22351
fwy        187310
river       31209
dtype: int64

<em> Wee see that most accidents happen on interstate highways (i-), so trying to predict accidents for these </em>

In [80]:
train_str_acc_df = train_all_df.query('quarter33==1 | quarter34==1 | quarter35==1 | quarter36==1 \
                                | quarter37==1 | quarter38==1 | quarter39==1 | quarter40==1 | quarter41==1')
train_str_i_acc_df = train_str_acc_df.loc[train_str_acc_df['i-']==1]
print(len(train_str_i_acc_df))
train_str_non_i_acc_df = train_str_acc_df.loc[train_str_acc_df['i-']==0]
#also taking same number of records where accidents did not happen on interstate highways for training
train_str_non_i_acc_df = train_str_non_i_acc_df.sample(len(train_str_i_acc_df))
train_str_acc_df = pd.concat([train_str_i_acc_df,train_str_non_i_acc_df])
print(len(train_str_acc_df))
train_str_acc_df = train_str_acc_df.sort_index()

1445
2890


In [81]:
test_str_acc_df = test_all_df.query('quarter33==1 | quarter34==1 | quarter35==1 | quarter36==1 \
                                | quarter37==1 | quarter38==1 | quarter39==1 | quarter40==1 | quarter41==1')
test_str_i_acc_df = test_str_acc_df.loc[test_str_acc_df['i-']==1]
test_str_non_i_acc_df = test_str_acc_df.loc[test_str_acc_df['i-']==0]
test_str_non_i_acc_df = test_str_non_i_acc_df.sample(len(test_str_i_acc_df))
test_str_acc_df = pd.concat([test_str_i_acc_df,test_str_non_i_acc_df])
print(len(test_str_acc_df))
test_str_acc_df = test_str_acc_df.sort_index()

556


In [82]:
train_str_acc_df.to_csv('../data_files/train_etc_str_acc.csv',index=False)
test_str_acc_df.to_csv('../data_files/test_etc_str_acc.csv',index=False)